In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv("/kaggle/input/credit-risk-dataset/credit_risk_dataset.csv")

df.head(2)

In [ ]:
df.describe()

In [ ]:
sns.countplot(x= df['loan_status'])

In [ ]:
graph = px.scatter_matrix(df, dimensions=['person_age','person_income','loan_amnt'], color= 'loan_status')
graph.show()

In [ ]:
df.isnull().sum()

In [ ]:
df.loc[df['loan_int_rate'].isnull(), 'loan_int_rate'] = df['loan_int_rate'].mean()
df.loc[df['person_emp_length'].isnull(), 'person_emp_length'] = df['person_emp_length'].mean()

In [ ]:
df.isnull().sum()

In [ ]:
new_order = [col for col in df.columns if col != 'loan_status'] + ['loan_status']
df = df[new_order]


In [ ]:
df.shape

In [ ]:
X_credit = df.iloc[:, 1:11].values

In [ ]:
y_credit = df.iloc[:, 11].values

In [ ]:
X_credit[0], y_credit, df.columns

In [ ]:
encoder_home_ownership = LabelEncoder()
encoder_loan_intent = LabelEncoder()
encoder_loan_grade = LabelEncoder()
encoder_cb_personal_default = LabelEncoder()

In [ ]:
X_credit[:, 1] = encoder_home_ownership.fit_transform(X_credit[:, 1])
X_credit[:, 3] = encoder_loan_intent.fit_transform(X_credit[:, 3])
X_credit[:, 4] = encoder_loan_grade.fit_transform(X_credit[:, 4])
X_credit[:, 8] = encoder_cb_personal_default.fit_transform(X_credit[:, 8])

In [ ]:
onehotencoder_credit = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [1, 3, 4, 8])], remainder='passthrough')

In [ ]:
X_credit = onehotencoder_credit.fit_transform(X_credit)

In [ ]:
X_credit

In [ ]:
scaler = StandardScaler()
X_credit = scaler.fit_transform(X_credit)

In [ ]:
X_credit[0]

In [ ]:
X_training, X_test, y_training, y_test = train_test_split(X_credit, y_credit, test_size= 0.2, random_state=0)

In [ ]:
X_training.shape, y_training.shape

# Naïve Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
naive_bayer = GaussianNB()
naive_bayer.fit(X_training, y_training)

In [ ]:
predict = naive_bayer.predict(X_test)
predict

In [ ]:
accuracy_score(y_test, predict)

In [ ]:
cm = ConfusionMatrix(naive_bayer)
cm.fit(X_training, y_training)
cm.score(X_test, y_test)

# Decision Tree

In [ ]:
decision_tree = DecisionTreeClassifier(criterion='entropy', random_state = 0)
decision_tree.fit(X_training, y_training)

In [ ]:
predict = decision_tree.predict(X_test)
predict

In [ ]:
accuracy_score(y_test, predict)

In [ ]:
cm = ConfusionMatrix(decision_tree)
cm.fit(X_training, y_training)
cm.score(X_test, y_test)

# Random Forest

In [ ]:
random_forest = RandomForestClassifier(n_estimators=150, criterion='entropy', random_state =0)
random_forest.fit(X_training, y_training)

In [ ]:
predict = random_forest.predict(X_test)
predict

In [ ]:
accuracy_score(y_test, predict)

In [ ]:
cm = ConfusionMatrix(random_forest)
cm.fit(X_training, y_training)
cm.score(X_test, y_test)